In [1]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from sklearn.model_selection import train_test_split
import textwrap

from torch.utils.data import Dataset,DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer

from transformers import(
    AdamW,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

In [2]:
from datasets import Dataset

In [3]:
def get_device_and_set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:1" if use_cuda else "cpu")
    return device
    
SEED = 123
device = get_device_and_set_seed(SEED)

In [4]:
device

device(type='cuda', index=1)

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [6]:
next(model.parameters()).is_cuda

True

In [7]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")  

In [8]:
tokenizer.max_model_input_sizes

{'t5-small': 512, 't5-base': 512, 't5-large': 512, 't5-3b': 512, 't5-11b': 512}

In [9]:
labels = tokenizer(
        'tôi thích bạn', max_length=256, truncation=True, padding=True
    )

In [10]:
labels

{'input_ids': [671, 1470, 1113, 1], 'attention_mask': [1, 1, 1, 1]}

## Prepare Data

In [11]:
train_path = '../processed_data/train_df.csv'

In [12]:
test_path = '../processed_data/test_df.csv'

In [13]:
test_df = pd.read_csv(test_path)
test_df.head()

,example_id,statement,law_id,article_id
0,qr6S2jA9GG,"Nếu không phạm tội quả tang, một người sẽ khôn...",Hiến pháp 2013,20
1,zFWTqve74q,"Tháng hành động quốc gia phòng, chống bạo lực ...","Luật Phòng, chống bạo lực gia đình 2022",7
2,bv98C2F6I9,Hoạt động thanh tra phải được thực hiện theo k...,Luật Thanh tra 2022,46
3,Kzmq9qdScq,Loại hợp đồng nào sau đây không được BLDS 2015...,Bộ Luật Dân sự 2015,402
4,8x8IdMONw9,Bản án phúc thẩm được tuyên vào ngày 20/01/202...,Luật Tố tụng hành chính 2015,242


In [14]:
train_df = pd.read_csv(train_path)
train_df.head()

,example_id,label,statement,law_id,article_id,segment
0,q9zjh7Uw7Q,No,Người xem dưới 16 tuổi được xem phim có nội du...,Luật Điện ảnh 2022,32,Người xem dưới 16 tuổi được xem phim có nội_du...
1,ckQFn8y202,No,"Trong vòng 03 ngày làm việc, kể từ ngày người ...","Luật Phòng, chống ma túy 2021",30,"Trong vòng 03 ngày làm_việc , kể từ ngày người..."
2,3ROu621ZEO,Yes,Viên chức có 02 năm liên tiếp bị phân loại đán...,Luật Viên chức 2010,29,Viên_chức có 02 năm liên_tiếp bị phân_loại đán...
3,VT1QuVmhCc,Yes,Các biện pháp cai nghiện ma túy là những biện ...,"Luật Phòng, chống ma túy 2021",28,Các biện_pháp cai_nghiện ma_tuý là những biện_...
4,0MwITLtbmg,No,Viên chức thuộc một đơn vị sự nghiệp công lập ...,Luật Viên chức 2010,14,Viên_chức thuộc một đơn_vị sự_nghiệp công_lập ...


In [15]:
legal_df = pd.read_csv('../processed_data/legal_df.csv')

In [16]:
merge_df = pd.merge(train_df, legal_df, on=['law_id', 'article_id'], suffixes=('_df1', '_df2'), how='outer')
merge_df

,example_id,label,statement,law_id,article_id,segment,text
0,q9zjh7Uw7Q,No,Người xem dưới 16 tuổi được xem phim có nội du...,Luật Điện ảnh 2022,32,Người xem dưới 16 tuổi được xem phim có nội_du...,Phân loại phim\n\n1. Phim được phân loại theo ...
1,bu3nyKAUNT,No,Người xem từ 18 tuổi trở lên được pháp xem phi...,Luật Điện ảnh 2022,32,Người xem từ 18 tuổi trở lên được pháp xem phi...,Phân loại phim\n\n1. Phim được phân loại theo ...
2,aS4Oxqxklj,Yes,Người từ 16 tuổi trở lên được phép xem phim có...,Luật Điện ảnh 2022,32,Người từ 16 tuổi trở lên được phép xem phim có...,Phân loại phim\n\n1. Phim được phân loại theo ...
3,E3nezW9AK0,No,Phim loại C là phim được phép phổ biến đến ngư...,Luật Điện ảnh 2022,32,Phim loại C là phim được phép phổ_biến đến ngư...,Phân loại phim\n\n1. Phim được phân loại theo ...
4,ckQFn8y202,No,"Trong vòng 03 ngày làm việc, kể từ ngày người ...","Luật Phòng, chống ma túy 2021",30,"Trong vòng 03 ngày làm_việc , kể từ ngày người...","Cai nghiện ma túy tự nguyện tại gia đình, cộng..."
...,...,...,...,...,...,...,...
2265,NaN,NaN,NaN,Luật Thanh niên 2020,36,NaN,Nội dung quản lý nhà nước về thanh niên\n\n1. ...
2266,NaN,NaN,NaN,Luật Thanh niên 2020,38,NaN,Trách nhiệm của Bộ Nội vụ\n\nBộ Nội vụ chịu tr...
2267,NaN,NaN,NaN,Luật Thanh niên 2020,39,NaN,"Trách nhiệm của các Bộ, cơ quan ngang Bộ\n\nCá..."
2268,NaN,NaN,NaN,Luật Thanh niên 2020,40,NaN,"Trách nhiệm của Hội đồng nhân dân, Ủy ban nhân..."


In [17]:
merge_df_test = pd.merge(test_df, legal_df, on=['law_id', 'article_id'], suffixes=('_df1', '_df2'), how='outer')
merge_df_test
merge_df_test = merge_df_test[~merge_df_test['statement'].isnull()]
merge_df_test

,example_id,statement,law_id,article_id,text
0,qr6S2jA9GG,"Nếu không phạm tội quả tang, một người sẽ khôn...",Hiến pháp 2013,20,1. Mọi người có quyền bất khả xâm phạm về thân...
1,zFWTqve74q,"Tháng hành động quốc gia phòng, chống bạo lực ...","Luật Phòng, chống bạo lực gia đình 2022",7,"Tháng hành động quốc gia phòng, chống bạo lực ..."
2,RwHiqOG8vb,"Tháng hành động quốc gia phòng, chống bạo lực ...","Luật Phòng, chống bạo lực gia đình 2022",7,"Tháng hành động quốc gia phòng, chống bạo lực ..."
3,wv4eBTDVKS,"Tháng hành động quốc gia phòng, chống bạo lực ...","Luật Phòng, chống bạo lực gia đình 2022",7,"Tháng hành động quốc gia phòng, chống bạo lực ..."
4,DyWakZCVVK,"Tháng hành động quốc gia phòng, chống bạo lực ...","Luật Phòng, chống bạo lực gia đình 2022",7,"Tháng hành động quốc gia phòng, chống bạo lực ..."
...,...,...,...,...,...
135,IdIDVZpLW4,Khoảng thời gian xảy ra trở ngại khách quan là...,Bộ Luật Dân sự 2015,156,Thời gian không tính vào thời hiệu khởi kiện v...
136,TiFpkKZtFM,Phán quyết trọng tài không bị xem xét lại theo...,Luật Trọng tài thương mại 2010,4,Nguyên tắc giải quyết tranh chấp bằng Trọng tà...
137,0XKeP3929n,Kinh phí hoạt động của cơ quan thanh tra do ng...,Luật Thanh tra 2022,112,Kinh phí hoạt động của cơ quan thanh tra; chế ...
138,zw1vs0Odbf,"Theo Luật Giáo dục năm 2019, một trong những y...",Luật Giáo dục 2019,7,"Yêu cầu về nội dung, phương pháp giáo dục\n\n1..."


In [18]:
merge_df = merge_df[~merge_df['statement'].isnull()]
merge_df

,example_id,label,statement,law_id,article_id,segment,text
0,q9zjh7Uw7Q,No,Người xem dưới 16 tuổi được xem phim có nội du...,Luật Điện ảnh 2022,32,Người xem dưới 16 tuổi được xem phim có nội_du...,Phân loại phim\n\n1. Phim được phân loại theo ...
1,bu3nyKAUNT,No,Người xem từ 18 tuổi trở lên được pháp xem phi...,Luật Điện ảnh 2022,32,Người xem từ 18 tuổi trở lên được pháp xem phi...,Phân loại phim\n\n1. Phim được phân loại theo ...
2,aS4Oxqxklj,Yes,Người từ 16 tuổi trở lên được phép xem phim có...,Luật Điện ảnh 2022,32,Người từ 16 tuổi trở lên được phép xem phim có...,Phân loại phim\n\n1. Phim được phân loại theo ...
3,E3nezW9AK0,No,Phim loại C là phim được phép phổ biến đến ngư...,Luật Điện ảnh 2022,32,Phim loại C là phim được phép phổ_biến đến ngư...,Phân loại phim\n\n1. Phim được phân loại theo ...
4,ckQFn8y202,No,"Trong vòng 03 ngày làm việc, kể từ ngày người ...","Luật Phòng, chống ma túy 2021",30,"Trong vòng 03 ngày làm_việc , kể từ ngày người...","Cai nghiện ma túy tự nguyện tại gia đình, cộng..."
...,...,...,...,...,...,...,...
71,InehZtAZn9,Yes,Viên chức bị kỷ luật từ khiển trách đến cách c...,Luật Viên chức 2010,56,Viên_chức bị kỷ_luật từ khiển_trách đến cách_c...,Các quy định khác liên quan đến việc kỷ luật v...
72,3lEnngVd8Z,Yes,Viên chức bị khiển trách thì thời hạn nâng lươ...,Luật Viên chức 2010,9,Viên_chức bị khiển_trách thì thời_hạn nâng lươ...,Đơn vị sự nghiệp công lập và cơ cấu tổ chức qu...
73,6nfjyV5thx,No,"Cơ quan nhà nước không có thẩm quyền theo dõi,...","Luật Phòng, chống ma túy 2021",14,Cơ_quan nhà_nước không có thẩm_quyền theo_dõi ...,"Kiểm soát hoạt động vận chuyển chất ma túy, ti..."
74,TisBomZhjP,Yes,Cách chức là một trong các hình thức xử lý kỷ ...,Luật Viên chức 2010,52,Cách_chức là một trong các hình_thức xử_lý kỷ_...,Các hình thức kỷ luật đối với viên chức \n\n1....


In [19]:
merge_df['inputs'] = 'hypothesis: ' + merge_df['statement'] + ' premise: '+ merge_df['text']

/tmp/ipykernel_2611414/529061005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_df['inputs'] = 'hypothesis: ' + merge_df['statement'] + ' premise: '+ merge_df['text']


In [20]:
merge_df_test['inputs'] = 'hypothesis: ' + merge_df_test['statement'] + 'premise: '+ merge_df_test['text']

In [21]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=1024, truncation=True, padding=True
    )
    
    
    labels = tokenizer(
        examples["labels"], max_length=256, truncation=True, padding=True
    )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [22]:
dict_obj = {'inputs': merge_df['inputs'], 'labels': merge_df['label']}
dataset = Dataset.from_dict(dict_obj)
train_data = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
len(train_data.__getitem__(1)['input_ids'])

893

In [25]:
len(train_data)

76

In [26]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")


training_args = Seq2SeqTrainingArguments("tmp/",
                                      do_train=True,
                                      do_eval=False,
                                      num_train_epochs=15,
                                      learning_rate=1e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=2,
                                      per_device_eval_batch_size=4,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      save_total_limit=3,
                                      #eval_steps=1,
                                      #evaluation_strategy="steps",
                                      # evaluation_strategy="no",
                                      fp16=True,
                                      )

## Training

In [30]:
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_data,
#     data_collator=data_collator,
# )

# trainer.train()

## Inference

In [20]:
from datasets import load_metric
metric = load_metric("rouge")

/data2/nghiatl/.tmp/ipykernel_253278/2721381364.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [31]:
model = AutoModelForSeq2SeqLM.from_pretrained("./tmp/checkpoint-285")

loading configuration file ./tmp/checkpoint-285/config.json
Model config T5Config {
  "_name_or_path": "./tmp/checkpoint-285",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "torch_dtype": "float32",
  "transformers_version": "4.26.0",
  "use_cache": true,
  "vocab_size": 36096
}

loading weights file ./tmp/checkpoint-285/pytorch_model.bin
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "t

In [32]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [24]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

In [25]:
import tqdm
import torch 
import numpy as np
metrics = load_metric('rouge')

max_target_length = 256
dataloader = torch.utils.data.DataLoader(test_data, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(dataloader):
  outputs = model.generate(
      input_ids=batch['input_ids'].to('cuda'),
      max_length=max_target_length,
      attention_mask=batch['attention_mask'].to('cuda'),
  )
  outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

  labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
  actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
  predictions.extend(outputs)
  references.extend(actuals)
  metrics.add_batch(predictions=outputs, references=actuals)


metrics.compute()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'rouge1': AggregateScore(low=Score(precision=0.9988018200762616, recall=0.998607608406491, fmeasure=0.9986347304154567), mid=Score(precision=0.9993805976766871, recall=0.9992200939966306, fmeasure=0.9992122872988795), high=Score(precision=0.999738926576217, recall=0.999624013478762, fmeasure=0.9995694423257271)),
 'rouge2': AggregateScore(low=Score(precision=0.9981314400993173, recall=0.9978736809435135, fmeasure=0.9978791822185674), mid=Score(precision=0.9989075108628183, recall=0.9987212911235258, fmeasure=0.9986936873389385), high=Score(precision=0.9995065952824334, recall=0.9993792675356922, fmeasure=0.9992694402121775)),
 'rougeL': AggregateScore(low=Score(precision=0.9988835018178597, recall=0.9986260973663208, fmeasure=0.9987033673961049), mid=Score(precision=0.9993805976766871, recall=0.9992294049835948, fmeasure=0.9992270898974809), high=Score(precision=0.9997396027312231, recall=0.9996311519021017, fmeasure=0.9995801538881147)),
 'rougeLsum': AggregateScore(low=Score(precisi

In [26]:
correct = 0
correct += sum(o==a for o, a in zip(predictions, references))
correct

5341

In [27]:
correct/ len(predictions)

0.9945996275605214

In [30]:
a= next(iter(dataloader))

In [31]:
tokenizer.decode(a['input_ids'][0], skip_special_tokens=True)

'nguyễn văn tiến thì dạ bên không cho'

In [52]:
idx = 3
merge_df_test.iloc[idx]['inputs']

'hypothesis: Tháng hành động quốc gia phòng, chống bạo lực gia đình được diễn ra vào mùa hè trong nămpremise: Tháng hành động quốc gia phòng, chống bạo lực gia đình\n\n1. Tháng hành động quốc gia phòng, chống bạo lực gia đình được tổ chức vào tháng 6 hằng năm để thúc đẩy hoạt động phòng, chống bạo lực gia đình và tôn vinh giá trị gia đình. \n\n2. Bộ Văn hóa, Thể thao và Du lịch chủ trì, phối hợp với cơ quan, tổ chức có liên quan để chỉ đạo, hướng dẫn và tổ chức thực hiện Tháng hành động quốc gia phòng, chống bạo lực gia đình.'

In [53]:
t = merge_df_test.iloc[idx]['inputs']
b = tokenizer(t, return_tensors='pt')
b

{'input_ids': tensor([[12957,  2938, 12508, 35862,  1867,   235,   256,   405,   254,   464,
         35790,   871,  1855,   374,   254,   721,    74,   769,   170,   162,
          1630,  2369,    80,   128,  3738,   428,  4918, 35862,  1867,   235,
           256,   405,   254,   464, 35790,   871,  1855,   374,   254,   721,
            40, 35792,  1867,   235,   256,   405,   254,   464, 35790,   871,
          1855,   374,   254,   721,    74,   655,   311,   162,   287,   163,
          4307,   128,   180,  1451,  1646,   758,   256,   464, 35790,   871,
          1855,   374,   254,   721,    39,  1973,  3937,   774,   747,   254,
           721, 35792,    60, 35792,   656,   652,   837, 35790,  3811,  2622,
            39,  2497,  1114,   436,  1760, 35790,  1271,   373,   123,   316,
           206, 35790,   655,   311,    71,   483,   206,   180,   337,   679,
         35790,  1184,   861,    39,   655,   311,   305,   219,  1867,   235,
           256,   405,   254,   464, 3

In [54]:
b['input_ids'].shape

torch.Size([1, 132])

In [55]:
outputs = model.generate(
      input_ids=b['input_ids'].to('cuda:1'),
      max_length=20,
      attention_mask=b['attention_mask'].to('cuda:1'),
  )
outputs

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}



tensor([[    0, 19074,     1]], device='cuda:1')

In [56]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Yes'